In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
 import pandas as pd
 from datasets import load_dataset, Dataset
 from transformers import T5Tokenizer,T5ForConditionalGeneration,Trainer,TrainingArguments,TrainerCallback
 import os
 from google.colab import drive
 import numpy as np
 import torch
 import matplotlib.pyplot as plt
 import seaborn as sn
 drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
dataset=load_dataset('paws',"labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [4]:
df=pd.DataFrame(dataset['train'])
df1=pd.DataFrame(dataset['validation'])
df2=pd.DataFrame(dataset['test'])

In [7]:
df=pd.concat([df,df1,df2])

In [8]:
df.shape

(65401, 4)

In [10]:
df=df[df.label == 1]
df.shape

(28904, 4)

In [16]:
def Preprocess_paws(dataset,label=1):
  df_l=pd.DataFrame(dataset)
  df_l=df_l[df_l.label == label]

  df_l['input_text']="paraphrase"+df_l['sentence1']
  df_l['target_text']=df_l['sentence2']

  return df_l[['input_text','target_text']]

In [17]:
train_df=Preprocess_paws(dataset['train'])
test_df=Preprocess_paws(dataset['test'])
val_df=Preprocess_paws(dataset['validation'])

In [18]:
test_df.head()

,input_text,target_text
2,"paraphraseIn January 2011 , the Deputy Secreta...","In January 2011 , FIBA Asia deputy secretary g..."
5,"paraphraseDuring her sophomore , junior and se...","During her second , junior and senior summers ..."
7,paraphraseHis father emigrated to Missouri in ...,His father emigrated to Missouri in 1868 but r...
9,paraphraseIt is situated south of Köroğlu Moun...,It is situated south of Köroğlu - mountains an...
10,paraphraseThe Río Blanco mine is a large coppe...,The Río Blanco - Mine is a large copper mine i...


In [21]:
train_dataset=Dataset.from_pandas(train_df)
test_dataset=Dataset.from_pandas(test_df)
val_dataset=Dataset.from_pandas(val_df)

In [22]:
train_dataset

Dataset({
    features: ['input_text', 'target_text', '__index_level_0__'],
    num_rows: 21829
})

In [24]:
model_name="t5-base"
tokenizer=T5Tokenizer.from_pretrained(model_name)
model=T5ForConditionalGeneration.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [25]:
max_length=max([len(tokenizer.encode(text)) for text in train_dataset['input_text']])
max_length

81

In [28]:
def tokenize_function(example):
  inputs=tokenizer(example['input_text'],max_length=max_length,padding='max_length',truncation=True)
  targets=tokenizer(example['target_text'],max_length=max_length,padding='max_length',truncation=True)
  inputs['labels']=targets['input_ids']
  return inputs

In [32]:
train_dataset=train_dataset.map(tokenize_function,batched=True)
val_dataset=val_dataset.map(tokenize_function,batched=True)
test_dataset=test_dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/21829 [00:00<?, ? examples/s]

Map:   0%|          | 0/3539 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

In [33]:
train_dataset[0]

{'input_text': 'paraphraseThe NBA season of 1975 -- 76 was the 30th season of the National Basketball Association .',
 'target_text': 'The 1975 -- 76 season of the National Basketball Association was the 30th season of the NBA .',
 '__index_level_0__': 1,
 'input_ids': [3856,
  27111,
  634,
  14512,
  774,
  13,
  16312,
  1636,
  3,
  3959,
  47,
  8,
  604,
  189,
  774,
  13,
  8,
  868,
  21249,
  2125,
  3,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [34]:
# Set directories in Google Drive
results_dir = "/content/drive/MyDrive/results"
model_dir = "/content/drive/MyDrive/saved_t5_model"

# Create the directories if they don't exist
os.makedirs(results_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [35]:

# Define training arguments
training_args = TrainingArguments(
    output_dir= results_dir,  # Directory where the results (logs, model outputs) will be saved
    overwrite_output_dir=True,  # Overwrite the output directory if it exists
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    eval_steps=500,  # Evaluate every 500 steps (this might not be necessary if evaluation is at epoch level)
    logging_dir='./logs',  # Directory to save logs
    logging_steps=100,  # Log every 100 steps
    learning_rate=3e-5,  # Learning rate for optimizer
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    num_train_epochs=5,  # Number of epochs to train the model
    weight_decay=0.01,  # L2 regularization to prevent overfitting
    save_strategy="no",  # Disable saving checkpoints during training to save memory
    fp16=True,  # Use mixed precision training to speed up training and reduce memory usage
    report_to="none"  # Disable reporting to avoid unwanted logging to external platforms
)

trainer = Trainer(
    model=model,  # The model to be trained
    args=training_args,  # The training arguments defined above
    train_dataset=train_dataset,  # The training dataset
    eval_dataset=val_dataset,  # The validation dataset
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.190100,0.171763
2,0.166100,0.165004
3,0.151800,0.163077
4,0.141600,0.163353
5,0.141900,0.163612


TrainOutput(global_step=13645, training_loss=0.177326800037802, metrics={'train_runtime': 3325.0844, 'train_samples_per_second': 32.825, 'train_steps_per_second': 4.104, 'total_flos': 1.05149246605056e+16, 'train_loss': 0.177326800037802, 'epoch': 5.0})

In [36]:
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('/content/drive/MyDrive/saved_t5_model/tokenizer_config.json',
 '/content/drive/MyDrive/saved_t5_model/special_tokens_map.json',
 '/content/drive/MyDrive/saved_t5_model/spiece.model',
 '/content/drive/MyDrive/saved_t5_model/added_tokens.json')

In [37]:

import os
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer from the saved directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

In [41]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer from the saved directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Preprocessing function for inference
def preprocess_input(sentence):
    return "paraphrase: " + sentence

# Generate paraphrases with corrected num_beams and num_return_sequences
def generate_paraphrase(input_text, model, tokenizer, max_length=max_length, num_beams=5, num_return_sequences=4, top_k=100, top_p=0.9, temperature=1.0):
    # Preprocess input
    input_text = preprocess_input(input_text)

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length, padding="max_length")

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate paraphrases
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length + 20,  # Increase max_length for more room in paraphrases
        num_beams=10,
        num_return_sequences=num_return_sequences,
        top_k=top_k,              # Use top-k sampling for diversity
        top_p=top_p,              # Use top-p sampling for nucleus-based sampling
        temperature=temperature,  # Encourage more exploratory generation
        do_sample=True,           # Enable sampling for top-k and top-p
        early_stopping=True
    )

    # Decode generated outputs
    paraphrased_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_texts

# Example sentence
input_sentence = ""

# Generate paraphrases
paraphrased_sentences = generate_paraphrase(
    input_sentence, model, tokenizer,max_length, num_return_sequences=4
)

# Display results
print(f"Original: {input_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: Barak obama was the prime minister of USA and he serve well
Paraphrase 1: Barak obama was the prime minister of the USA and he served well.
Paraphrase 2: Barak obama was the prime minister of the USA and he serves well.
Paraphrase 3: Barak Obama was the prime minister of the USA and he served well.
Paraphrase 4: Barak obama was the prime minister of the USA and he serve well.
